# Exercise 1

Work on this before the next lecture on 10 April. We will talk about questions, comments, and solutions during the exercise after the second lecture.

Please do form study groups! When you do, make sure you can explain everything in your own words, do not simply copy&paste from others.

The solutions to a lot of these problems can probably be found with Google. Please don't. You will not learn a lot by copy&pasting from the internet.

If you want to get credit/examination on this course please upload your work to **your GitHub repository** for this course **before** the next lecture starts and post a link to your repository [in this thread](https://github.com/wildtreetech/advanced-comp-2017/issues/1). If you worked on things together with others please add their names to the notebook so we can see who formed groups.


## Objective

There are two objectives for this set of exercises:

* get you started using python, scikit-learn, matplotlib, and GitHub. You will be using them a lot during the course, so make sure you get a good foundation to build on.

* working through the steps of opening a new dataset, plotting the data, fitting a model to it, evaluating your model, and deciding on model complexity.

## Question 0

Install python, scikit-learn (v0.18), matplotlib, jupyter and git.

Instructions for doing so: https://github.com/wildtreetech/advanced-comp-2017/blob/master/install.md

Documentation and guides for the various tools:

* [jupyter quickstart](http://jupyter.readthedocs.io/en/latest/content-quickstart.html)
* [try jupyter without installing anything](https://try.jupyter.org/)
* [matplotlib homepage](http://matplotlib.org/)
* [matplotlib gallery](http://matplotlib.org/gallery.html)
* [scikit-learn homepage](http://scikit-learn.org/stable/)
* [scikit-learn examples](http://scikit-learn.org/stable/auto_examples/index.html)
* [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
* [try git online without installing anything](https://try.github.io/levels/1/challenges/1)


### GitHub and git

* [Create a GitHub account]() for yourself or use one you already have.
* Follow the guide on [creating a new repository](https://help.github.com/articles/create-a-repo/). Name the repository "advanced-comp-2017".

Read up on `git clone`, `git pull`, `git push`, `git add` and `git commit`. Once you master these five commands you should be good for this course. There is a whole universe of complex things that `git` can do for you, don't worry about them for now. Once you feel comfortable with the basics you can always step it up later.

---

These are some useful default imports for plotting and [`numpy`](http://www.numpy.org/)

In [ ]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import numpy as np
np.random.seed(123)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state

---

## Question 1

In the lecture we used the nearest neighbour classifier to classify points from a toy dataset into either "red" or "blue" classes. We investigated how the performance changes as a function of model complexity and what this means for the performance of our classifier on unseen data. Instead of using a linear model as in the lecture, use a k-nearest neighbour model.

* plot your dataset
* split your dataset into a training and testing set. Comment on how you decided to split your data.
* evaluate the performance of the classifier on your training dataset.
* evaluate the performance of the classifier on your testing dataset.
* repeat the above two steps for varying splits (10-90, 20-80, 30-70, ...) and comment
  on what you see. Is there a "best" way to split your data?
* comment on why the two performance estimates agree or disagree.
* plot the accuracy of the classifier as a function of `n_neighbors`.
* comment on the similarities and differences between the performance on the testing and training dataset.
* is a KNeighbor Classifier with 4 or 10 neighbors more complicated?
* find the best setting of `n_neighbors` for this dataset.
* why is this the best setting?

Use `make_blobs(n_samples=400, centers=23, random_state=42)` to create a simple dataset and use the `KNeighborsClassifier` classifier to answer the above questions.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier

labels = ["b", "r"]
X, y = make_blobs(n_samples=400, centers=23, random_state=42)
y = np.take(labels, (y < 10))

In [ ]:
# Your solution


### plot dataset

In [ ]:
plt.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
plt.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')

### Split into training and validation data

A procedure to avoid bias is to randomly shuffle the data, then pick a subset. I will use 20% of data for validation, which I believe is a decent compromise between having enough data for validation and having enough data for training.

In [ ]:
rng = np.random.RandomState(42)

def train_test_split(X,y,train_size):
    
    XX = np.c_[X,y]
    np.random.shuffle(XX)

    X_train = XX[:int(train_size*X.shape[0]),:-1]
    y_train = XX[:int(train_size*X.shape[0]),-1]
    X_val   = XX[X_train.shape[0]:,:-1]
    y_val   = XX[X_train.shape[0]:,-1]

    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = train_test_split(X,y,0.8)

print(X.shape)
print(X_train.shape)
print(X_val.shape)

In [ ]:
# make sure everything was ok
plt.subplot(1,3,1)
plt.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
plt.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')
plt.subplot(1,3,2)
plt.plot(X_train[y_train==labels[0],0],X_train[y_train==labels[0],1],'*b')
plt.plot(X_train[y_train==labels[1],0],X_train[y_train==labels[1],1],'*r')
plt.subplot(1,3,3)
plt.plot(X_val[y_val==labels[0],0],X_val[y_val==labels[0],1],'*b')
plt.plot(X_val[y_val==labels[1],0],X_val[y_val==labels[1],1],'*r')

### evaluate the performance of the classifier on training and validation dataset.

In [ ]:
accuracies_test = []
accuracies_train = []
ks = np.arange(1, 25, 1)

for n in range(50):
    X, y = make_blobs(n_samples=400, centers=23, random_state=42+n)
    y = np.take(labels, (y < 10))
    X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=0.5)
    train_scores = []
    test_scores = []
    for k in ks:
        clf = KNeighborsClassifier(n_neighbors=k)
        clf.fit(X_train, y_train)
        train_scores.append(clf.score(X_train, y_train))
        test_scores.append(clf.score(X_val, y_val))
        
    accuracies_test.append(test_scores)
    accuracies_train.append(train_scores)
    
    plt.plot(ks, train_scores, c='b', alpha=0.1)
    plt.plot(ks, test_scores, c='r', alpha=0.1)
    
plt.plot(ks, np.array(accuracies_test).mean(axis=0), label='Test', c='r', lw=4)
plt.plot(ks, np.array(accuracies_train).mean(axis=0), label='Train', c='b', lw=4)
plt.xlabel('k or inverse model complexity')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.xlim((0, max(ks)))
plt.ylim((0.5, 1.));

In [ ]:
accuracies_test = []
accuracies_train = []
ks = np.arange(1, 25, 1)

for idx, training_size in enumerate([0.9, 0.8, 0.7, 0.6]):

    plt.subplot(2,2,idx+1)
    
    for n in range(50):
        X, y = make_blobs(n_samples=400, centers=23, random_state=42+n)
        y = np.take(labels, (y < 10))
        X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=training_size)
        train_scores = []
        test_scores = []
        for k in ks:
            clf = KNeighborsClassifier(n_neighbors=k)
            clf.fit(X_train, y_train)
            train_scores.append(clf.score(X_train, y_train))
            test_scores.append(clf.score(X_val, y_val))

        accuracies_test.append(test_scores)
        accuracies_train.append(train_scores)

        plt.plot(ks, train_scores, c='b', alpha=0.1)
        plt.plot(ks, test_scores, c='r', alpha=0.1)


    plt.plot(ks, np.array(accuracies_test).mean(axis=0), label='Test', c='r', lw=4)
    plt.plot(ks, np.array(accuracies_train).mean(axis=0), label='Train', c='b', lw=4)
    plt.xlabel('k or inverse model complexity')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.xlim((0, max(ks)))
    plt.ylim((0.5, 1.));

### Discussion

Training and Testing errors are almost always different, and quite often the testing error is larger compared to the training error.

In any case, a large difference between training and testing errors is not a good sign w.r.t the goodness of the model. 
- If the training error is very small but the testing error large,we are probably in a region of overfitting. We can try to simplify the model, or dedicate a larger chunk of the data to validation.
- If the training error is very large, on the other hand, it may be that we are in a region of underfitting. We might require a more complex model, or a more different model altogether.

It is obviously difficult to define a 'perfect' proportion for splitting training and validation data. In this particular example, we see that pretty large training datasets (90-10 or 80-20) appear to lead to better results for both training and validation errors. However, I don't know how general this rule can be.  

### Complexity of the model

The KNearestNeighbor classifier uses the $k$ closest neighbours of a point to predict its value. To understand how this relates to complexity, I find it easier to think of two extremes:
1. $k == dataset size$: in this case the value predicted at any point would be equal to the average of all the known values in the dataset. This corresponds to the __simplest__ model possible;
2. $k=1$: in this case, the value predicted is equal to the value of the closes neighbor. This corresponds to the __most complex__ model, in the sense that it predicts piecewise constant function with potentially the largest number of discontinuities.

In this sense, a model with $k=4$ is more complex than a model with $k=10$.

### Data splitting

To find the best value of k for this dataset, I refer to the plots above, and in particular the one relating to the 80-20 split (the top right plot), which appears to give best accuracy overall.

### Best setting

Very large values of k are correlated to small values of the training error. However, there seems to be an inverted U shape for the testing error, which reaches its maximum accuracy (for validation error) roughly around k=15. However, there is also a question of model complexity to take into account: the inverted U shape we see is actually quite flat, so in principle one could achieve comparable performance with a much smaller value of k, e.g. around 10 or even a bit less. 

---

## Question 2

This is a regression problem. It mostly follows the setup of the classification problem so you should be able to reuse some of your work.

* plot your dataset
* fit a kNN regressor with varying number of `n_neighbors` and compare each regressors predictions to the location of the training and testing points. 
* plot the mean squared error of the classifier as a function of `n_neighbors` for both training and testing datasets.
* comment on the similarities and differences between the performance on the testing and training dataset.
* find the best setting of `n_neighbors` for this dataset.
* why is this the best setting?
* can you explain why the mean square error on the training dataset plateaus between ~`n_neihgors`=5 to 15 at the value that it does?

Use `make_regression()` to create the dataset and use `KNeighborsRegressor` to answer the above questions. Take a look at scikit-learn's [`metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) module to compute the mean squared error.

In [ ]:
def make_regression(n_samples=100, noise_level=0.8, random_state=2):
    rng = check_random_state(random_state)
    X = np.linspace(-2, 2, n_samples)
    y = 2.0 * X + np.sin(5 * X) + rng.randn(n_samples) * noise_level
    #y = 2.0*X + rng.randn(n_samples) * noise_level
    #y = rng.randn(n_samples) * noise_level
    
    return X.reshape(-1, 1), y

In [ ]:
X, y = make_regression(100,0.8,42)

### Plot dataset

In [ ]:
plt.plot(X,y,'*')
X_, y_ = make_regression(400,0.0,42)
plt.plot(X_, y_, '-r')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

X, y = make_regression(400,0.8,2)
X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=0.8)
print(X_train.shape)
print(X_val.shape)

rgr = KNeighborsRegressor(n_neighbors=1)
rgr.fit(X_train, y_train)

plt.plot(X_train,y_train,'b*')
plt.plot(X_val,y_val,'*r')
plt.plot(X_val, rgr.predict(X_val), 'k*')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

accuracies_test = []
accuracies_train = []
ks = np.arange(1, 35, 1)

for n in range(50):
    X, y = make_regression(100,0.8,2)
    X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=0.8)
    train_scores = []
    test_scores = []
    for k in ks:
        rgr = KNeighborsRegressor(n_neighbors=k)
        rgr.fit(X_train, y_train)
        train_scores.append( mean_squared_error(y_train, rgr.predict(X_train)) )
        test_scores.append( mean_squared_error(y_val, rgr.predict(X_val)) )
        
    accuracies_test.append(test_scores)
    accuracies_train.append(train_scores)
    
    plt.plot(ks, train_scores, c='b', alpha=0.1)
    plt.plot(ks, test_scores, c='r', alpha=0.1)
    
plt.plot(ks, np.array(accuracies_train).mean(axis=0), '*-', label='Train', c='b', lw=4, alpha=0.8)
plt.plot(ks, np.array(accuracies_test).mean(axis=0), '*-', label='Test', c='r', lw=4, alpha=0.8)
plt.xlabel('k or inverse model complexity')
plt.ylabel('MSE')
plt.legend(loc='best')
plt.xlim((0, max(ks)))
plt.ylim((-0.05, np.max(np.array(accuracies_test))))

### Discussion


##### Reason for plateau
For intermediate values of $k$ ($5 < k < 15$), we notice a plateau in the training error close to a value of $0.6$. Given that we defined a noise level of $\sigma = 0.8$, it is tempting to associate the plateau value to $\sigma^2$.
The hypothesis is therefore that, for those special values of $k$, the training error approximates the noise in the data. It is reasonable to think that, since our target function $y = 2X + sin(5X)$ does not have very large derivatives, it can be approximated locally to a constant function. We confirm this hypothesis by trying to perform regression on a constant function (i.e. only noise), code below. Indeed, for increasing $k$ the training error settles on a value equal to $\sigma^2$.
We conclude that the plateau in the training error is given by two factors:
1. $k$ is small enough, so that the considered neighbours span a small enough length that allows to consider the target function *almost constant*
2. $k$ is large enough that the training error settles on the value of the noise in the data.

This can be seen as a consequence of the error decomposition:
$ error = bias^2 + var^2 + \sigma^2 $
where for $5 < k < 15$ we have minimal bias and variance on the training error.

In [ ]:
def make_regression(n_samples=100, noise_level=0.8, random_state=2):
    rng = check_random_state(random_state)
    X = np.linspace(-2, 2, n_samples)
    #y = 2 * X + np.sin(5 * X) + rng.randn(n_samples) * noise_level
    y = rng.randn(n_samples) * noise_level
    
    
    return X.reshape(-1, 1), y

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

accuracies_test = []
accuracies_train = []
ks = np.arange(1, 20, 1)

for n in range(50):
    X, y = make_regression(100,0.8,2)
    X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=0.8)
    train_scores = []
    test_scores = []
    for k in ks:
        rgr = KNeighborsRegressor(n_neighbors=k)
        rgr.fit(X_train, y_train)
        train_scores.append( mean_squared_error(y_train, rgr.predict(X_train)) )
        test_scores.append( mean_squared_error(y_val, rgr.predict(X_val)) )
        
    accuracies_test.append(test_scores)
    accuracies_train.append(train_scores)
    
    plt.plot(ks, train_scores, c='b', alpha=0.1)
    plt.plot(ks, test_scores, c='r', alpha=0.1)
    
plt.plot(ks, np.array(accuracies_train).mean(axis=0), '*-', label='Train', c='b', lw=4, alpha=0.8)
plt.plot(ks, np.array(accuracies_test).mean(axis=0), '*-', label='Test', c='r', lw=4, alpha=0.8)
plt.xlabel('k or inverse model complexity')
plt.ylabel('MSE')
plt.legend(loc='best')
plt.xlim((0, max(ks)))
plt.ylim((-0.05, np.max(np.array(accuracies_test))))

---

## Question 3

Logistic regression. Use a linear model to solve a two class classification problem.

* What is the difference between a linear regression model and a logistic regression model?
* plot your data and split it into a training and test set
* draw your guess for where the decision boundary will be on the plot. Why did you pick this one?
* use the `LogisticRegression` classifier to fit a model to your training data
* extract the fitted coefficients from the model and draw the fitted decision boundary
* create a function to draw the decision surface (the classifier's prediction for every point in space)
* why is the boundary where it is?
* **(bonus)** create new datasets with increasingly larger amounts of noise (increase the `cluster_std` argument) and plot the decision boundary for each case. What happens and why?
* create 20 new datasets by changing the `random_state` parameter and fit a model to each. Visualise the variation in the fitted parameters and the decision boundaries you obtain. Is this a high or low variance model?

Use `make_two_blobs()` to create a simple dataset and use the `LogisticRegression` classifier to answer the above questions.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def make_two_blobs(n_samples=400, cluster_std=2., random_state=42):
    rng = check_random_state(random_state)
    X = rng.multivariate_normal([5,0], [[cluster_std**2, 0], [0., cluster_std**2]],
                                size=n_samples//2)
    
    X2 = rng.multivariate_normal([0, 5.], [[cluster_std**2, 0], [0., cluster_std**2]],
                                 size=n_samples//2)
    X = np.vstack((X, X2))
    return X, np.hstack((np.ones(n_samples//2), np.zeros(n_samples//2)))

X, y = make_two_blobs()
labels = ['b', 'r']
y = np.take(labels, (y < 0.5))
print(X.shape)
print(y.shape)

In [ ]:
# Your answer


### Logistic Regression

is a technique that gives a model for predicting an output $y$ as a function of an input $y = f(x)$. 
The difference with regular regression is that $y$ is, in logistic regression, a categorical variable, namely $y \in \{ 0,1 \}$, whereas in regular regression $y$ is a real variable $y \in \mathbb{R}$.

In [ ]:
plt.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
plt.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')

In [ ]:
def train_test_split(X,y,train_size):
    
    XX = np.c_[X,y]
    np.random.shuffle(XX)

    X_train = XX[:int(train_size*X.shape[0]),:-1]
    y_train = XX[:int(train_size*X.shape[0]),-1]
    X_val   = XX[X_train.shape[0]:,:-1]
    y_val   = XX[X_train.shape[0]:,-1]

    return X_train, X_val, y_train, y_val

In [ ]:
X_train,X_val, y_train,y_val = train_test_split(X, y, train_size=0.8)

## Decision Boundary

#### my guess for the decision boundary

was found visually, trying to put a line that would minimise the number of misplaced points. Also, I was tempted by a sort of *corridor of white* between the two blobs, which made me think the decision boundary should pass through there.

In [ ]:
plt.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
plt.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')
xx = np.linspace(start=-6.0, stop=10.0, num=200)
guessed_dec_boundary= 2.6/3.0*xx
plt.plot(xx,guessed_dec_boundary,'-k')

#### fitting the model

In [ ]:
from sklearn.linear_model import LogisticRegression

ls = LogisticRegression()
fit = ls.fit(X,y)
print(fit.coef_)

In [ ]:
plt.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
plt.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')

xx = np.linspace(start=-6.0, stop=10.0, num=200)

guessed_dec_boundary= 2.6/3.0*xx
plt.plot(xx,guessed_dec_boundary,'--k', alpha=0.3)

actual_decision_boundary = - fit.coef_[0][0]*xx / fit.coef_[0][1]
plt.plot(xx,actual_decision_boundary,'-k')

### Drawing the decision surface

The position of the decision surface is determined by how we convert the linear model into a class probability (and obviously also by the linearity of the underlying model). 
In particular, logistic regression aims at minimizing the cost function:

$ L(\mathbf{w}) =  \sum_n log( 1 + e^{ \mathbf{x}^T_n\mathbf{w} } ) - y_n \mathbf{x}^T_n\mathbf{w} $

The decision boundary is therefore in this particular position because it is the linear function that minimizes $L(\mathbf{w})$.

It should be noted, for example, that it doesn't explicitly minimize the total number of misclassified points, because in $L(\mathbf{w})$ not all misclassifications have the same importance.

In [ ]:
def evaluate_decision(fit, xx, yy):
    assert xx.shape == yy.shape
    print(xx.shape)
    zz = np.zeros(shape=xx.shape)
    coefs = fit.coef_[0]
    for row in range(xx.shape[0]):
        for col in range(xx.shape[1]):
            zz[row][col] = ( np.inner( np.array([xx[0][row], yy[col][0]]), coefs) < 0 ) 
            
    return zz
        
x_space = np.linspace(start=np.min(X[:,0]), stop=np.max(X[:,0]), num=400)
y_space = np.linspace(start=np.min(X[:,1]), stop=np.max(X[:,1]), num=400)
xx,yy = np.meshgrid(x_space, y_space)   
    
def plot_decision_surface(datapoints, fit, ax):
    X = datapoints[:,0]
    Y = datapoints[:,1]
    x_space = np.linspace(start=np.min(X), stop=np.max(X), num=400)
    y_space = np.linspace(start=np.min(Y), stop=np.max(Y), num=400)
    xx,yy = np.meshgrid(x_space, y_space)
    zz = evaluate_decision(fit, xx, yy)
    zz.reshape(xx.shape)
    ax.contourf(xx, yy, zz, alpha=0.6, interp='None', cmap=plt.cm.RdBu_r)

fig = plt.figure()
ax = fig.add_subplot(111)

plot_decision_surface(X,fit,ax)
ax.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
ax.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')

### Bonus

As the level of noise increases, the decision surface retains the same shape. This is probably because the centroids of the blobs actually didn't change.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

num_iter = 4
noise = 1.
labels = ['b', 'r']
ls = LogisticRegression()
fig = plt.figure()
    
def plot_decision_surface2(datapoints, fit, ax, xmin, xmax, ymin, ymax):
    X = datapoints[:,0]
    Y = datapoints[:,1]
    x_space = np.linspace(start=xmin, stop=xmax, num=400)
    y_space = np.linspace(start=ymin, stop=ymax, num=400)
    xx,yy = np.meshgrid(x_space, y_space)
    zz = evaluate_decision(fit, xx, yy)
    zz.reshape(xx.shape)
    ax.contourf(xx, yy, zz, alpha=0.6, interp='None', cmap=plt.cm.RdBu_r)
    
for iter in range(num_iter):
    noise *= 1.5

    X, y = make_two_blobs(n_samples=400, cluster_std=noise, random_state=42)
    y = np.take(labels, (y < 0.5))
    
    fit = ls.fit(X,y)

    ax = fig.add_subplot(2,2,iter+1)

    plot_decision_surface2(X,fit,ax, -10, 10, -10, 10)
    ax.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
    ax.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')
    ax.set_xlim([-10,10])
    ax.set_ylim([-10,10])
    

### Using different random seeds

I am kind of surprised that using different random seeds produces exactly the same coefficients, up to many decimal places!
This is definitely a low-variance method.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import random

num_iter = 20
labels = ['b', 'r']
ls = LogisticRegression()
fig = plt.figure()
    
fits = []
    
random.seed(42)

for iter in range(num_iter):
    rnd_seed = random.randint(1, 230)
    
    X, y = make_two_blobs(n_samples=200, cluster_std=2., random_state=rnd_seed)
    y = np.take(labels, (y < 0.5))
    
    fit = ls.fit(X,y)
    fits.append(fit)

    ax = fig.add_subplot(5,4,iter+1)

    plot_decision_surface(X,fit,ax)
    ax.plot(X[y==labels[0],0],X[y==labels[0],1],'*b')
    ax.plot(X[y==labels[1],0],X[y==labels[1],1],'*r')


In [ ]:
plt.figure()

xcoefs = [ f.coef_[0][0] for f in fits ]
ycoefs = [ f.coef_[0][1] for f in fits ]

print(xcoefs)

plt.scatter(xcoefs, ycoefs)

---

## Question 4

Logistic regression. Use a more complex linear model to create a two class classifier for the "circle inside a circle" problem. Think about how you can increase the complexity of a logistic regression model. Visualise the classificatio naccuracy as a function of the model complexity.

Use `make_circles(n_samples=400, factor=.3, noise=.1)` to create a simple dataset and use the `LogisticRegression` classifier to answer the above question.

In [ ]:
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=400, factor=.3, noise=.1)
labels = ['b', 'r']
y = np.take(labels, (y < 0.5))

plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
# Your answer


In [ ]:
def transform_coordinates(X):
    Z = np.zeros(shape=X.shape)
    for idx,coord in enumerate(X):
        x = coord[0]
        y = coord[1]
        rho = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y,x)
        Z[idx] = [rho, theta]
        
    return Z

Z = transform_coordinates(X)
plt.scatter(Z[:,0], Z[:,1], c=y)

In [ ]:
from sklearn.linear_model import LogisticRegression

ls = LogisticRegression()
fit = ls.fit(Z,y)
print(fit.coef_)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

plot_decision_surface(Z,fit,ax)
plt.scatter(Z[:,0], Z[:,1], c=y)

### Completely separable

Logistic regression cannot converge if the data, as is the case after transformations, is completely separable (see for example [this Quora answer](https://www.quora.com/Why-does-logistic-regression-not-converge-when-the-data-is-completely-separable). A simple way to look at this is to think: how would the algorithm decide between two decision boundaries that are both able to split the data with 100% accuracy, but are slightly different?

A simple way around this would be to generate a lot of data, until at some point we get because of the randomness at least one point that invalidates the completely separable property. This might not be feasible or too expensive.

A better way would be to add a regularization term. However in this case it seems that this is nto enough.

In [ ]:
ls = LogisticRegression(penalty='l2',C=1e-15)
fit = ls.fit(Z,y)
fig = plt.figure()
ax = fig.add_subplot(111)

plot_decision_surface(Z,fit,ax)
plt.scatter(Z[:,0], Z[:,1], c=y)